In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
# Setting a random seed
np.random.seed(42)

In [3]:
# Loading in the dataset
df = pd.read_csv('../data/US_Accidents_Dec21_updated.csv')

# Data Cleaning

In [4]:
# Initial columns present in dataset
df.columns

Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

## Dropping Columns

In [5]:
df.drop(['Number', 'Zipcode', 'Wind_Chill(F)', 'Airport_Code', 'Nautical_Twilight', 'Astronomical_Twilight',
        'Weather_Timestamp', 'ID', 'End_Time', 'End_Lat', 'End_Lng', 'Description', 
         'Country', 'Civil_Twilight', 'Distance(mi)'], axis = 1, inplace = True)

**Reasoning**
- Number, representing street number, is too specific of a focus for our purposes
- There's no discernible relationship between Zipcode and frequency of accidents or severity of accident
- Wind Chill has a highly correlated with temperature and wind speed, with both of the other features having a higher correlation with our target variable, Severity.
- Airport Code isn't relevant to our analysis
- Nautical Twilight and Astronomical Twilight are similar to our chosen day/night metric (day/night based on Sunrise/Sunset)
- Weather Timestamp isn't relevant to our analysis
- ID of accident case isn't relevant to our analysis
- End_Time, End_Lat, End_Lng aren't meaningful for exploring how to reduce accident severity (should be exploring the start time, latitude, longitude of accidents)
- Description of the accident isn't relevant to our analysis 
- Country is the same across the entire corpus as it details accidents occurring in the U.S.
- Civil Twilight is similar to our chosen day/night metric (day/night based on Sunrise/Sunset)
- Distance(mi) or length of the road affected by the accident isn't meaningful for exploring the reduction of accident severity as it tracks a value obtained as a result of the accident

## Addressing Missing Values

In [6]:
# Determining all columns with null values
nulls = df.isna().sum().sort_values(ascending = False)
nulls = nulls[nulls.values > 0]
nulls

Precipitation(in)    549458
Wind_Speed(mph)      157944
Wind_Direction        73775
Humidity(%)           73092
Weather_Condition     70636
Visibility(mi)        70546
Temperature(F)        69274
Pressure(in)          59200
Timezone               3659
Sunrise_Sunset         2867
City                    137
Street                    2
dtype: int64

In [7]:
# Exploring null precipitation values per weather condition category
pd.DataFrame(df['Precipitation(in)'].isnull().groupby(
    by = df['Weather_Condition']).sum().sort_values(ascending = False)).head(20)

,Precipitation(in)
Weather_Condition,
Clear,172786
Overcast,72357
Mostly Cloudy,72029
Partly Cloudy,51271
Fair,45103
Scattered Clouds,44052
Cloudy,8959
Haze,6675
Light Rain,3908


Based off domain knowledge, the 6 weather condition categories with the most null values should have low or zero precipitation. As such, we've imputed their null precipitation values with zero.

In [8]:
df.loc[(df['Precipitation(in)'].isna()) & (df['Weather_Condition'].str.contains(
    'Clear|Overcast|Mostly Cloudy|Partly Cloudy|Fair|Scattered Clouds', regex = True)), ['Precipitation(in)']] = 0

We decided to impute the remaining null precipitation values with each weather category's median precipitation, as it's robust to outliers and the precipitation distributions per weather category were largely skewed.

In [9]:
df['Precipitation(in)'].fillna(df.groupby('Weather_Condition')['Precipitation(in)'].transform(
    'median'), inplace = True)

In [10]:
df['Precipitation(in)'].isna().sum() / df.shape[0]

0.020438316378136617

Some other precipitation values were null, which we decided to drop entirely as they only constitute a small percentage (~2.04%) of our dataset. 

In [11]:
df.dropna(subset = 'Precipitation(in)', inplace = True)

In [12]:
# Exploring null wind speeds per wind direction category
wind_speed_nulls = df['Wind_Speed(mph)'].isnull().groupby(by = df['Wind_Direction']).sum().sort_values(
    ascending = False)
wind_speed_nulls[wind_speed_nulls.values > 0]

Wind_Direction
Calm     76022
North     2907
Name: Wind_Speed(mph), dtype: int64

The calm wind direction category has the most null wind speed values. Given our intuition that calm winds should have 0 wind speeds, we imputed the nulls with 0.

In [13]:
df.loc[(df['Wind_Speed(mph)'].isna()) & 
       (df['Wind_Direction'].str.contains('Calm', regex = True)), ['Wind_Speed(mph)']] = 0

In [14]:
df['Wind_Speed(mph)'].isna().sum() / df.shape[0]

0.009287138147839328

We decided to drop the remaining null wind speed rows as they constitute a very small percentage (~0.093) of our dataset. 

In [15]:
df.dropna(subset = 'Wind_Speed(mph)', inplace = True)

In [16]:
# Exploring what columns might be correlated to Humidity to support imputation process
df[['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 
    'Wind_Speed(mph)', 'Precipitation(in)']].corr()

,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
Temperature(F),1.000000,-0.366206,0.137220,0.210497,0.086516,-0.004348
Humidity(%),-0.366206,1.000000,0.139746,-0.363129,-0.181569,0.074751
Pressure(in),0.137220,0.139746,1.000000,0.035240,-0.054383,0.006759
Visibility(mi),0.210497,-0.363129,0.035240,1.000000,0.037998,-0.114270
Wind_Speed(mph),0.086516,-0.181569,-0.054383,0.037998,1.000000,0.022316
Precipitation(in),-0.004348,0.074751,0.006759,-0.114270,0.022316,1.000000


In [17]:
# Number of rows with null humidity compared to entire df
13151 / df.shape[0]

0.004762606638967183

We dropped null humidities as they constitute a very small percentage of data (~0.048%) and lack of domain knowledge.


In [18]:
df.dropna(subset = 'Humidity(%)', inplace = True)

In [19]:
# Examining the remaining columns/features with nulls
temp = df.isna().sum().sort_values(ascending = False)
temp = temp[temp.values > 0]
temp

Visibility(mi)       7949
Weather_Condition    6600
Sunrise_Sunset       2256
Pressure(in)         2001
City                  132
Wind_Direction         30
Street                  1
dtype: int64

In [20]:
temp.index

Index(['Visibility(mi)', 'Weather_Condition', 'Sunrise_Sunset', 'Pressure(in)',
       'City', 'Wind_Direction', 'Street'],
      dtype='object')

We dropped the remaining null values as they all constitute a very small percentage of our dataset.

In [21]:
df.dropna(subset = ['Visibility(mi)', 'Weather_Condition', 'Sunrise_Sunset',
       'Pressure(in)', 'City', 'Wind_Direction', 'Street'], inplace = True)

### Dropping remaining irrelevant columns

We dropped the wind direction category as we weren't interested in exploring it for our project.

In [22]:
df.drop(['Wind_Direction'], axis = 1, inplace = True)

In [23]:
# Exporting cleaned df to csv
df.to_csv('../data/cleaned_df.csv')